### Setup

In [1]:
print('ViT')
print('patch 16')
print('damage 0.5')

ViT
patch 16
damage 0.5


In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]='4'

In [3]:
import cv2
import skimage.exposure
from numpy.random import default_rng
import random
import numpy as np
import skimage
import matplotlib.pyplot as plt
import tensorflow as tf
from vit_keras import vit
from tensorflow import keras
# from tensorflow.python.keras.engine.keras_tensor import KerasTensor


C:\Users\shubh\anaconda3\Lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
C:\Users\shubh\anaconda3\Lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.12.0 and strictly below 2.15.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.16.1 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure yo

ModuleNotFoundError: No module named 'keras.src.engine'

In [4]:
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [5]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


### Prepare the data

In [6]:
img_w = 4200
img_h = 420
channels = 3
num_classes = 130
batch_size = 16 #32

In [7]:
def noise_function(image):
    image = image / 255.0
#     image = skimage.util.random_noise(image, mode='gaussian', var=1)
#     image = skimage.util.random_noise(image, mode='s&p', amount=1)
    image = skimage.util.random_noise(image, mode='speckle', var=1)
    image = image * 255
    return image

In [8]:
def damage_function(image): 
    patch_size = 32
    num_patches = img_h // patch_size * img_w // patch_size

    for i in range(num_patches):
        prob = 0.5
        if random.choices([0, 1], [1-prob, prob])[0]:
            image[(i//(img_w // patch_size))*patch_size:(i//(img_w // patch_size))*patch_size+patch_size, (i%(img_w // patch_size))*patch_size:(i%(img_w // patch_size))*patch_size+patch_size, :] = np.zeros([patch_size, patch_size, channels])
    return image

In [9]:
def random_damage_function(image):
    image = image.astype(np.uint8)
    height, width = image.shape[:2]

    # define random seed to change the pattern
    rng = default_rng()

    # create random noise image
    noise = rng.integers(0, 255, (height,width), np.uint8, True)

    # blur the noise image to control the size
    blur = cv2.GaussianBlur(noise, (0,0), sigmaX=15, sigmaY=15, borderType = cv2.BORDER_DEFAULT)

    # stretch the blurred image to full dynamic range
    stretch = skimage.exposure.rescale_intensity(blur, in_range='image', out_range=(0,255)).astype(np.uint8)

    # threshold stretched image to control the size
    thresh = cv2.threshold(stretch, 175, 255, cv2.THRESH_BINARY)[1]

    # apply morphology open and close to smooth out and make 3 channels
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (9,9))
    mask = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    mask = cv2.merge([mask,mask,mask])

    # add mask to input
    result = cv2.add(image, mask)
    return result

In [10]:
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    preprocessing_function = damage_function
)    

train_ds = image_generator.flow_from_directory(batch_size=batch_size,
                                                directory= "C:/Users/shubh/OneDrive/Desktop/IITP/Expeeriment Set3/train",
                                                shuffle=True,
                                                target_size=(img_h, img_w),
                                                class_mode = 'categorical')

# validation_ds = image_generator.flow_from_directory(batch_size=batch_size,
#                                                     directory='../../datasets/val',
#                                                     shuffle=False,
#                                                     target_size=(img_h, img_w),
#                                                     class_mode = 'categorical')

test_ds = image_generator.flow_from_directory(batch_size=batch_size,
                                                directory="C:/Users/shubh/OneDrive/Desktop/IITP/Expeeriment Set3/On_Off",
                                                shuffle=False,
                                                target_size=(img_h, img_w),
                                                class_mode = 'categorical')

Found 1300 images belonging to 130 classes.
Found 260 images belonging to 130 classes.


In [11]:
patch_size = 16
num_patches = img_h // patch_size * img_w // patch_size

In [12]:
class Patches(tf.keras.layers.Layer):
    def __init__(self, patch_size):
        super(Patches, self).__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches

In [13]:
image = next(train_ds)[0][0]/255.0
fig = plt.figure(figsize=(10, 10))
fig.patch.set_facecolor('black')
plt.imshow(image)
plt.axis("off")
# plt.savefig('./sample_data.png')
plt.show()
plt.close()

resized_image = tf.image.resize(tf.convert_to_tensor([image]), size=(img_h, img_w))
patches = Patches(patch_size)(resized_image)
print(f"Image size: {img_h} X {img_w}")
print(f"Patch size: {patch_size} X {patch_size}")
print(f"Patches per image: {patches.shape[1]}")
print(f"Elements per patch: {patches.shape[-1]}")

n = img_w // patch_size
fig = plt.figure(figsize=(10, 10))
fig.patch.set_facecolor('black')
plt.subplots_adjust(left=0.1,
                bottom=0.1,
                right=0.9,
                top=0.9,
                wspace=0.3,
                hspace=0.3)
for i, patch in enumerate(patches[0]):
    ax = plt.subplot(n, n, i + 1)
    patch_img = tf.reshape(patch, (patch_size, patch_size, channels))
    plt.imshow(patch_img)
    plt.axis("off")
# plt.savefig('./patches.png')
plt.show()
plt.close()

ValueError: could not broadcast input array from shape (32,32,3) into shape (4,32,3)

In [17]:
input_shape = (img_h, img_w, channels)

In [18]:
def create_vit_classifier():
    input = tf.keras.layers.Input(shape=input_shape)
    x = vit.preprocess_inputs(input)
    base_model = vit.build_model(
                image_size = (img_h, img_w),
                patch_size = patch_size,
                num_layers = 12,
                hidden_size = 768,
                num_heads = 12,
                name = 'vit',
                mlp_dim = 3072,
                classes = num_classes,
                dropout = 0.2,
                activation = 'softmax'
            )
    output = base_model(x)
    model = tf.keras.Model(inputs=input, outputs=output)
    return model

In [19]:
vit_classifier = create_vit_classifier()

NameError: name 'vit' is not defined

In [20]:
vit_classifier.summary()

NameError: name 'vit_classifier' is not defined

In [21]:
tf.keras.utils.plot_model(
    vit_classifier,
    to_file='./model.png',
    show_shapes=True,
    show_dtype=True,
    show_layer_names=True,
    rankdir='TB',
    expand_nested=True,
    dpi=96,
    layer_range=None,
    show_layer_activations=True
)

NameError: name 'vit_classifier' is not defined

### Compile, train, and evaluate the mode

In [ ]:
learning_rate = 0.00001
num_epochs = 300
checkpoint_filepath = f"../../weights/ViT_damage_0.5/{patch_size}/checkpoint"

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate = learning_rate)

vit_classifier.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
    metrics=[
        tf.keras.metrics.CategoricalAccuracy(name="accuracy"),
    ],
)

In [ ]:
def run_experiment(model):
    callbacks = [
        tf.keras.callbacks.ModelCheckpoint(checkpoint_filepath, monitor="val_accuracy", save_best_only=True, save_weights_only=True),
    ]

    history = model.fit(
        train_ds,
        epochs=num_epochs,
        validation_data=validation_ds,
        callbacks=callbacks,
    )

    return history

In [ ]:
history = run_experiment(vit_classifier)

### Results

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('ViT loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper right')
plt.savefig('./loss.png')
plt.show()
plt.close()

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('ViT accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='lower right')
plt.savefig('./accuracy.png')
plt.show()
plt.close()

In [ ]:
def model_evaluate(model, test_ds):
    model.load_weights(checkpoint_filepath)
    _, accuracy = model.evaluate(test_ds)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")
    with open('results.txt', 'w') as f:
        f.write(f"Test accuracy: {round(accuracy * 100, 2)}%")

In [ ]:
model_evaluate(vit_classifier, test_ds)

In [ ]:
while(True):
    print('ViT')
    print('patch 16')
    print('damage 0.5')